## New data processing

In [550]:
!pip3 install quadprog

  Using cached https://files.pythonhosted.org/packages/9a/bd/d39af77b8c3a0b8881b960c467272f29a6b8c70fb425e8c665abb81e0b82/Cython-0.29.19-cp37-cp37m-manylinux1_x86_64.whl
  Running setup.py bdist_wheel for quadprog ... done
  Stored in directory: /home/aims/.cache/pip/wheels/36/dd/b1/849989444c0a5930927b260663019b7da6cff864fc224c2747
Successfully built quadprog


In [100]:
# ----- Import Libraries and Datasets ------

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [101]:
# Import Datasets train and test
Train_data = pd.read_csv('Xtr.csv')
Test_data = pd.read_csv('Xte.csv')

print('Datasets have been loaded')

Datasets have been loaded


In [102]:
# Import label
Label_data = pd.read_csv('Ytr.csv')

In [103]:
print (" Training data: ",Train_data.shape)
print (" Test data: ",Test_data.shape)
print (" label: ",Label_data.shape)

 Training data:  (2000, 2)
 Test data:  (1000, 2)
 label:  (2000, 2)


In [104]:
Data = Train_data.drop(['Id'], axis = 1)
Data

,seq
0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...
1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...
2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...
3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...
4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...
...,...
1995,TAACTTTTGACAGGTCAGAATACAAAACTGATTTATTTACAGTGTC...
1996,ACGCCCATTCCGCCCTGCTAAGCCTCGCCCATTACATCCAGACTGC...
1997,TGGCTACTAGCTAGAGATAGCATCTCTCTGTGGACAACTCTCCAGC...
1998,CCCAGCTGTCAAAAAGCAGCCCAAAGGAAGCTCACGGTGTGCCGGC...


In [459]:
Test_data;

In [105]:
Label_data= Label_data.drop(['Id'], axis=1)

In [106]:
y_label= np.asarray(Label_data)
y_label = np.where(y_label ==0, -1, 1).squeeze(1)

In [107]:
# function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size=6):
    return [sequence[x:x+size] for x in range(len(sequence) - size + 1)]


In [464]:
def tokenize_data(data,k):
    seq = []
    for i in range(len(data)):
        word = data.iloc[i]['seq']
        word = getKmers(word, k)
        seq.append(word)
    names = ['txt'+str(i) for i in range(len(word))]
    seq = np.array(seq)
    return pd.DataFrame(data = seq,columns=names)

In [516]:
train_data_spe = tokenize_data(Train_data,k=3)
test_data_spe = tokenize_data(Test_data,k=3)

In [517]:
test_data_spe;

In [518]:
train_data_spe;

In [519]:
train_test = pd.concat([train_data_spe,test_data_spe])

In [520]:
train_test;

In [521]:
from sklearn.preprocessing import OneHotEncoder
def onehot(X):
    enc = OneHotEncoder(sparse=False)
    enc.fit(X)
    onehotlabels = enc.transform(X)
    return onehotlabels

In [522]:
train_test_oneHot = onehot(train_test)

In [523]:
train , test = train_test_oneHot[:2000,:] , train_test_oneHot[2000:,:]

In [524]:
# y = np.array(Label_data['Bound'])
# y

In [525]:
y = y_label
y

array([ 1, -1,  1, ...,  1,  1,  1])

In [526]:
train


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [527]:
y_label.shape

(2000,)

In [528]:
test.shape

(1000, 6168)

## Using Kernel Logistic Regression

In [529]:
def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the RBF kernel with parameter sigma
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    sigma: float
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis = -1)
    X1_norm = np.sum(X1 ** 2, axis = -1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

In [530]:
def linear_kernel(X1, X2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the linear kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return X1.dot(X2.T)

def quadratic_kernel(X1, X2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the quadratic kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return (1 + linear_kernel(X1, X2))**2

In [531]:
class KernelMethodBase(object):
    '''
    Base class for kernel methods models
    
    Methods
    ----
    fit
    predict
    '''
    kernels_ = {
        'linear': linear_kernel,
        'quadratic': quadratic_kernel,
        'rbf': rbf_kernel,
        #if you want to add your own kernel
        #'customer_kernel':custom_kernel
    }
    def __init__(self, kernel='linear', **kwargs):
        self.kernel_name = kernel
        self.kernel_function_ = self.kernels_[kernel]
        self.kernel_parameters = self.get_kernel_parameters(**kwargs)
        
    def get_kernel_parameters(self, **kwargs):
        params = {}
        if self.kernel_name == 'rbf':
            params['sigma'] = kwargs.get('sigma', None)
            
#         if self.kernel_name == 'customer_kernel':
#             params['parameter_1'] = kwargs.get('parameter_1', None)
#             params['parameter_2'] = kwargs.get('parameter_2', None)
        return params

    def fit(self, X, y, **kwargs):
        return self
        
    def decision_function(self, X):
        pass

    def predict(self, X):
        pass

In [532]:
class KernelRidgeRegression(KernelMethodBase):
    '''
    Kernel Ridge Regression
    '''
    def __init__(self, lambd=0.01, **kwargs):
        self.lambd = lambd
        # Python 3: replace the following line by
        # super().__init__(**kwargs)
        super(KernelRidgeRegression, self).__init__(**kwargs)

    def fit(self, X, y, sample_weights=None):
        n, p = X.shape
        assert (n == len(y))
    
        self.X_train = X
        self.y_train = y
        
        if sample_weights is not None:
            w_sqrt = np.sqrt(sample_weights)
            self.X_train = self.X_train * w_sqrt[:, None]
            self.y_train = self.y_train * w_sqrt
        # Hint the kernel matrix is computed by
        #K=self.kernet_function_(X, X, **self.kernel_parameters)
        
        A = self.kernel_function_(X, X, **self.kernel_parameters)
        A[np.diag_indices_from(A)] += n*self.lambd
        # self.alpha = (K + n lambda I)^-1 y
        self.alpha = np.linalg.solve(A , self.y_train)

        return self
    
    def decision_function(self, X):
        K_x = self.kernel_function_ (X, self.X_train, **self.kernel_parameters)
        return K_x.dot(self.alpha)
    
    def predict(self, X):
        return self.decision_function(X)
    
    
class WeightedKernelRidgeRegression(KernelRidgeRegression):
    '''
    Weighted Kernel Ridge Regression
    
    This is just used for the KernelLogistic following up
    '''
    def fit(self, K, y, sample_weights=None):

        self.y_train = y
        n = len(self.y_train)
        
        w = np.ones_like(self.y_train) if sample_weights is None else sample_weights
        W = np.diag(np.sqrt(w))
        
        A = W.dot(K).dot(W)
        A[np.diag_indices_from(A)] += self.lambd * n
        # self.alpha = W (K + n lambda I)^-1 W y
        self.alpha = W.dot(np.linalg.solve(A , W.dot(self.y_train)))

        return self

In [533]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class KernelLogisticRegression(KernelMethodBase):
    '''
    Kernel Logistic Regression
    '''
    def __init__(self, lambd=0.1, **kwargs):
        self.lambd = lambd
        # Python 3: replace the following line by
        # super().__init__(**kwargs)
        super(KernelLogisticRegression, self).__init__(**kwargs)

    def fit(self, X, y, max_iter=100, tol=1e-5):
        n, p = X.shape
        assert (n == len(y))
    
        self.X_train = X
        self.y_train = y
        
        K = self.kernel_function_(X, X, **self.kernel_parameters)
        
        # IRLS
        KRR = KernelRidgeRegression(
            lambd=2*self.lambd,
            kernel=self.kernel_name,
            **self.kernel_parameters
        )
        # Initialize
        alpha = np.zeros(n)
        # Iterate until convergence or max iterations
        for n_iter in range(max_iter):
            alpha_old = alpha
            m = K.dot(alpha_old)
            w = sigmoid(m)*sigmoid(-m)
            z = m + y/ sigmoid(y*m)
            alpha = KRR.fit(self.X_train, z, sample_weights= w).alpha
            
            # Break condition (achieved convergence)
            if np.sum((alpha-alpha_old)**2) < tol:
                break

        self.n_iter = n_iter
        self.alpha = alpha

        return self
            
    def decision_function(self, X_test):
        K_x = self.kernel_function_(X_test, self.X_train, **self.kernel_parameters)
        # probability of y=1(between o and 1)
        return sigmoid(K_x.dot(self.alpha))

    def predict(self, X):
        probas =self.decision_function(X)
        predicted_classes = np.where(probas < 0.5, -1,1)
        return predicted_classes
    
    def error(self, y_true, y_pred):
        e = (y_true== y_pred).mean()
        return e

In [534]:
from sklearn.model_selection import train_test_split

In [535]:
#X_train, X_test, y_train, y_test = train_test_split( train, y, test_size=0.33, random_state=42)

In [536]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class KernelLogisticRegression_weight(KernelMethodBase):
    '''
    Kernel Logistic Regression
    '''
    def __init__(self, lambd=0.1, **kwargs):
        self.lambd = lambd
        # Python 3: replace the following line by
        # super().__init__(**kwargs)
        super(KernelLogisticRegression_weight, self).__init__(**kwargs)

        
    def fit(self, X, y, max_iter=100, tol=1e-5):
    
        self.X_train = X
        self.y_train = y
        
        K = self.kernel_function_(X, X, **self.kernel_parameters)
        
        # IRLS
        WKRR = WeightedKernelRidgeRegression(
            lambd=self.lambd,
            kernel=self.kernel_name,
            **self.kernel_parameters
        )
        # Initialize
        alpha = np.zeros_like(self.y_train)
        # Iterate until convergence or max iterations
        for n_iter in range(max_iter):
            alpha_old = alpha
            f = K.dot(alpha_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*f)
            alpha = WKRR.fit(K, z, sample_weights=w).alpha
            # Break condition (achieved convergence)
            if np.sum((alpha-alpha_old)**2) < tol:
                break
        self.n_iter = n_iter
        self.alpha = alpha

        return self
            
    def decision_function(self, X_test):
        K_x = self.kernel_function_(X_test, self.X_train, **self.kernel_parameters)
        # probability of y=1(between o and 1)
        return sigmoid(K_x.dot(self.alpha))

    def predict(self, X):
        probas =self.decision_function(X)
        predicted_classes = np.where(probas < 0.5, -1,1)
        return predicted_classes
    
    def error(self, y_true, y_pred):
        e = (y_true== y_pred).mean()
        return e

In [537]:
# !pip install --quiet optuna

In [538]:
import optuna

def objective(trial):
    #iris = sklearn.datasets.load_iris()
    
    lambd = trial.suggest_int('lambd', 1e-1, 9)
    sigma = trial.suggest_int('sigma', 1e-1, 10)

    #max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
    
    kernel = 'quadratic'

    #sigma = 100
#     model = KernelSVM(C=C, kernel=kernel,sigma=sigma)
    error=[]
    
    for i, (train_index, validate_index) in enumerate(kfold.split(train)):
            X_train, y_train = train[train_index], y[train_index]
            X_valid, y_valid = train[validate_index], y[validate_index]
            model = KernelLogisticRegression(lambd=lambd, kernel=kernel, sigma=sigma)
            #y_pred = model.fit(X_train, np.where(y_train==0, -1,1)).predict(X_valid)
            y_pred = model.fit(X_train, y_train).predict(X_valid)
            #model.error(y_pred, y_valid)
            error.append(model.error(y_pred, y_valid))
    
    #clf = sklearn.ensemble.RandomForestClassifier(
     #   n_estimators=n_estimators, max_depth=max_depth)
    
    return np.mean(error)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-05-31 17:09:13,973] Finished trial#0 with value: 0.6529999999999999 with parameters: {'lambd': 6, 'sigma': 7}. Best is trial#0 with value: 0.6529999999999999.
[I 2020-05-31 17:09:16,620] Finished trial#1 with value: 0.6529999999999999 with parameters: {'lambd': 6, 'sigma': 9}. Best is trial#0 with value: 0.6529999999999999.
[I 2020-05-31 17:09:19,103] Finished trial#2 with value: 0.655 with parameters: {'lambd': 4, 'sigma': 6}. Best is trial#2 with value: 0.655.
[I 2020-05-31 17:09:21,457] Finished trial#3 with value: 0.658 with parameters: {'lambd': 2, 'sigma': 6}. Best is trial#3 with value: 0.658.
[I 2020-05-31 17:09:25,629] Finished trial#4 with value: 0.6595000000000001 with parameters: {'lambd': 0, 'sigma': 6}. Best is trial#4 with value: 0.6595000000000001.
[I 2020-05-31 17:09:28,461] Finished trial#5 with value: 0.6535 with parameters: {'lambd': 7, 'sigma': 3}. Best is trial#4 with value: 0.6595000000000001.
[I 2020-05-31 17:09:30,987] Finished trial#6 with value: 0.652

Accuracy: 0.663
Best hyperparameters: {'lambd': 1, 'sigma': 8}


# k-fold

In [539]:
## cross validation
from sklearn.model_selection import KFold
kfold=KFold(n_splits=5,shuffle=True,random_state=7)
error = []

kernel = 'quadratic'
sigma = 8.
lambd = 1.0

for i, (train_index, validate_index) in enumerate(kfold.split(train)):
            X_train, y_train = train[train_index], y[train_index]
            X_valid, y_valid = train[validate_index], y[validate_index]
            model = KernelLogisticRegression(lambd=lambd, kernel=kernel, sigma=sigma)
            y_pred = model.fit(X_train, y_train).predict(X_valid)
            #model.error(y_pred, y_valid)
            error.append(model.error(y_pred, y_valid))
            #print(y_valid)
            print(f'error fold {i}: {error[i]}')
        
print(f'\nAverage accuracy is : {np.mean(error)}\n')

error fold 0: 0.6425
error fold 1: 0.625
error fold 2: 0.655
error fold 3: 0.7125
error fold 4: 0.68

Average accuracy is : 0.663



In [541]:
np.sum(y_pred==-1)

160

# Training on the whole data

In [542]:
kernel = 'quadratic'
sigma = 8.
lambd = 1.

model = KernelLogisticRegression(lambd=lambd, kernel=kernel, sigma=sigma)
y_pred = model.fit(train, y).predict(test)

#print('Test error: {:.2%}'.format(model.error(y_pred, y_test)))

In [513]:
np.sum(y_pred==-1)

427

In [258]:
submission=[]
for i in range(len(y_pred)):
    if y_pred[i] == -1:
        submission.append(0)
    else:submission.append(1)
Id = np.array([i for i in range(len(y_pred))])
sub = {'Id':Id,'Bound':submission}
sub = pd.DataFrame(data = sub)
sub.to_csv('model_output_klrg_675.csv',index=False)

In [247]:
train.shape

(2000, 77088)

In [544]:
np.sum(y_pred==1)

576

In [260]:
y_pred;

In [ ]:
from sklearn.model_selection import KFold
kfold=KFold(n_splits=5)

In [1]:
# ## cross validation

# error = []

# for i, (train_index, validate_index) in enumerate(kfold.split(data_new)):
#             X_train, y_train = data_new[train_index], y_label[train_index]
#             X_valid, y_valid = data_new[validate_index], y_label[validate_index]
#             model = KernelLogisticRegression(lambd=lambd, kernel=kernel, sigma=sigma)
#             y_pred = model.fit(X_train, y_train).predict(X_valid)
#             #model.error(y_pred, y_valid)
#             error.append(model.error(y_pred, y_valid))
#             #print(y_valid)
#             print(f'error fold {i}: {error[i]}')
        
# print(f'\nAverage accuracy is : {np.mean(error)}\n')

## Kernel SVM

In [483]:
import cvxopt

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    #cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp = cvxopt_qp

In [484]:
def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)
        
    # Dual formulation, soft margin
    # P = np.diag(y) @ K @ np.diag(y)
    P = np.diag(y).dot(K).dot(np.diag(y))
    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis, :]
    A = A.astype('float')
    b = np.array([0.])
    return P, q, G, h, A, b

K = linear_kernel(X_train, X_train)
alphas = solve_qp(*svm_dual_soft_to_qp_kernel(K, y_train, C=1.))

In [545]:
class KernelSVM(KernelMethodBase):
    '''
    Kernel SVM Classification
    
    Methods
    ----
    fit
    predict
    '''
    kernels_ = {
        'linear': linear_kernel,
        'quadratic': quadratic_kernel,
        'rbf': rbf_kernel,
        # 'custom_kernel': custom_kernel, # Your kernel
    }
    def __init__(self, C=0.1, **kwargs):
        self.C = C
        # Python 3: replace the following line by
        # super().__init__(**kwargs)
        super(KernelSVM, self).__init__(**kwargs)

    def fit(self, X, y, tol=1e-3):
        n, p = X.shape
        assert (n == len(y))
    
        self.X_train = X
        self.y_train = y
        
        # Kernel matrix
        K = self.kernel_function_(
            self.X_train, self.X_train, **self.kernel_parameters)
        
        # Solve dual problem
        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=self.C))
        
        # Compute support vectors and bias b
        sv = np.logical_and((self.alpha > tol), (self.C - self.alpha > tol))
        self.bias = y[sv] - K[sv].dot(self.alpha * y)
        self.bias = self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]

        return self
        
    def decision_function(self, X):
        K_x = self.kernel_function_(X, self.X_train, **self.kernel_parameters)
        return K_x.dot(self.alpha * self.y_train) + self.bias

    def predict(self, X):
        return np.sign(self.decision_function(X))
    
    def error(self, y_true, y_pred):
        e = (y_true== y_pred).mean()
        return e

In [549]:
## cross validation
from sklearn.model_selection import KFold
kfold=KFold(n_splits=5,shuffle=True,random_state=7)
error = []

kernel = 'linear'
sigma = 1.
C = 100.

for i, (train_index, validate_index) in enumerate(kfold.split(train)):
            X_train, y_train = train[train_index], y[train_index]
            X_valid, y_valid = train[validate_index], y[validate_index]
            model = KernelSVM(C=C, kernel=kernel, sigma=sigma)
            y_pred = model.fit(X_train, y_train).predict(X_valid)
            #model.error(y_pred, y_valid)
            error.append(model.error(y_pred, y_valid))
            #print(y_valid)
            print(f'error fold {i}: {error[i]}')
        
print(f'\nAverage accuracy is : {np.mean(error)}\n')

error fold 0: 0.6175
error fold 1: 0.5875
error fold 2: 0.6175
error fold 3: 0.645
error fold 4: 0.625

Average accuracy is : 0.6185



In [434]:
## inference

kernel = 'linear'
sigma = 4.
C = 6.


model = KernelSVM(C=C, kernel=kernel, sigma=sigma)
y_pred_svm = model.fit(train, y).predict(test)


In [437]:
# submission

submission=[]
for i in range(len(y_pred_svm)):
    if y_pred_svm[i] == -1:
        submission.append(0)
    else:submission.append(1)
Id = np.array([i for i in range(len(y_pred_svm))])
sub = {'Id':Id,'Bound':submission}
sub = pd.DataFrame(data = sub)
sub.to_csv('model_output_svm_6735.csv',index=False)

In [436]:
np.sum(y_pred_svm ==-1)

420

In [382]:
y_pred_svm

In [451]:
import optuna

def objective(trial):
    #iris = sklearn.datasets.load_iris()
    
    C = trial.suggest_int('C', 1e-2, 11)
    sigma = trial.suggest_int('sigma', 1e-2, 10)

    #max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
    
    kernel = 'linear'

    #sigma = 100
#     model = KernelSVM(C=C, kernel=kernel,sigma=sigma)
    error=[]
    
    for i, (train_index, validate_index) in enumerate(kfold.split(train)):
            X_train, y_train = train[train_index], y[train_index]
            X_valid, y_valid = train[validate_index], y[validate_index]
            model = KernelSVM(C=C, kernel=kernel, sigma=sigma)
            y_pred = y_pred = model.fit(X_train, y_train).predict(X_valid)
            #model.error(y_pred, y_valid)
            error.append(model.error(y_pred, y_valid))
    
    #clf = sklearn.ensemble.RandomForestClassifier(
     #   n_estimators=n_estimators, max_depth=max_depth)
    
    return np.mean(error)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-05-30 23:57:45,424] Finished trial#0 with value: 0.6185 with parameters: {'C': 5, 'sigma': 4}. Best is trial#0 with value: 0.6185.
[I 2020-05-30 23:57:55,603] Finished trial#1 with value: 0.6185 with parameters: {'C': 10, 'sigma': 5}. Best is trial#0 with value: 0.6185.
[I 2020-05-30 23:58:05,070] Finished trial#2 with value: 0.6185 with parameters: {'C': 2, 'sigma': 5}. Best is trial#0 with value: 0.6185.
[I 2020-05-30 23:58:14,481] Finished trial#3 with value: 0.6185 with parameters: {'C': 2, 'sigma': 4}. Best is trial#0 with value: 0.6185.
[I 2020-05-30 23:58:24,522] Finished trial#4 with value: 0.6185 with parameters: {'C': 8, 'sigma': 3}. Best is trial#0 with value: 0.6185.
[I 2020-05-30 23:58:34,350] Finished trial#5 with value: 0.6185 with parameters: {'C': 4, 'sigma': 3}. Best is trial#0 with value: 0.6185.
/home/aims/.local/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: Mean of empty slice.
/home/aims/.local/lib/python3.7/site-packages/numpy/core

Accuracy: 0.6185
Best hyperparameters: {'C': 5, 'sigma': 4}
